# NextGEOSS
## Invoke the ITC's Multi-hazard Enhanced Risk Assessment service using JupyterLab

### Inspect the OGC WPS capabilities

In [2]:
from owslib.wps import WebProcessingService
from owslib.wps import monitorExecution
from owslib.wps import printInputOutput

In [2]:
wps_url = "http://ebv-lai.itc.nextgeoss.terradue.com/wps/WebProcessingService"

In [4]:
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

In [5]:
wps.getcapabilities()

In [6]:
for operation in wps.operations:
    print operation.name

GetCapabilities
DescribeProcess
Execute


In [7]:
for process in wps.processes:
    print process.identifier

com.terradue.wps_oozie.process.OozieAbstractAlgorithm


### Inspect the OGC WPS process information and inputs

In [8]:
process_id = 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm'
process = wps.describeprocess(process_id)

In [9]:
process.title

'LAI'

In [10]:
process.abstract

'Determine LAI for the Netherlands'

In [11]:
for input in process.dataInputs:
    printInputOutput(input)
    print '***'

 identifier=type, title=Product type, abstract=Select the Sentinel product type, data type=string
 Allowed Value: S2MSI2A
 Allowed Value: S2MSI2Ap
 Default Value: S2MSI2A 
 minOccurs=1, maxOccurs=1
***
 identifier=startdate, title=Start date, abstract=Start date of the Sentinel data (fe 2018-05-15), data type=string
 Any value allowed
 Default Value: 2018-05-04 
 minOccurs=1, maxOccurs=1
***
 identifier=enddate, title=End date, abstract=End date of the Sentinel data  (fe 2018-05-15), data type=string
 Any value allowed
 Default Value: 2018-05-08 
 minOccurs=1, maxOccurs=1
***
 identifier=cloud_cover, title=Cloud cover, abstract=Maximum cloud cover (in %), data type=string
 Any value allowed
 Default Value: 10 
 minOccurs=1, maxOccurs=1
***
 identifier=bounding_area, title=Area of interest, abstract=Specify a WKT polygon as boundary, data type=string
 Any value allowed
 Default Value: POLYGON((6.77 51.73,6.95 51.73,6.95 51.83,6.77 51.83,6.77 51.73)) 
 minOccurs=1, maxOccurs=100
***
 ide

### Prepare the execution request

In [15]:
inputs = []
inputs.append(('type','S2MSI2A'))
inputs.append(('startdate','2018-05-04'))
inputs.append(('enddate','2018-05-08'))
inputs.append(('cloud_cover','10'))
inputs.append(('bounding_area','POLYGON((6.77 51.73,6.95 51.73,6.95 51.83,6.77 51.83,6.77 51.73))'))
inputs.append(('storname','||'))


outputs = [('result_osd',True)]

### Execute and monitor the request

In [16]:
execution = wps.execute(process_id, inputs, outputs)

In [ ]:
monitorExecution(execution)

In [14]:
execution.isSucceded()

True

### Access the results' OpenSearch Description (osd)

In [15]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [16]:
results_osd = execution.processOutputs[0].reference

In [ ]:
print results_osd

In [ ]:
import requests
r = requests.get(results_osd)
r.text

In [ ]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
root.attrib['href']

### Perform an OpenSearch query for the results enclosures

In [20]:
import cioppy

In [21]:
ciop = cioppy.Cioppy()

In [22]:
search = ciop.search(end_point = root.attrib['href'],
                     params = [],
                     output_fields='enclosure')

In [23]:
for result in search:
    print result['enclosure']

### Download an output file

The following section can be leveraged when published results are available.

In [24]:
#search[0]['enclosure']

In [25]:
#r = requests.get(search[0]['enclosure'],stream=True)

In [26]:
#with open('output', 'wb') as fd:
#    for chunk in r.iter_content(chunk_size=128):
#        fd.write(chunk)